In [ ]:
!pip install transformers datasets accelerate evaluate torch torchvision

In [ ]:
!pip install --upgrade transformers datasets

In [ ]:
import os
import shutil
import requests

cache_dir = os.path.expanduser("~/.cache/huggingface/datasets")
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)

current_dir_cache = "./.cache"
if os.path.exists(current_dir_cache):
    print(f"현재 디렉토리 캐시 폴더 삭제: {current_dir_cache}")
    shutil.rmtree(current_dir_cache)

# 다른 필수 라이브러리 설치
!pip install transformers datasets accelerate evaluate torch torchvision scikit-learn

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import os
import shutil
import torch
import pandas as pd

model_name = "google/vit-base-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name)

# 모델의 레이블 정보 확인
print(f"모델의 클래스 수: {len(model.config.id2label)}")
print(f"모델의 첫 5개 클래스: {list(model.config.id2label.values())[:5]}...")

# classify_and_organize_food_images
def classify_and_organize_food_images(image_folder_path, output_base_path):

    # 출력 기본 경로가 없으면 생성
    if not os.path.exists(output_base_path):
        os.makedirs(output_base_path)

    # GPU 사용 가능 여부 확인
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    # 지정된 이미지 폴더 내의 모든 파일 순회
    for filename in os.listdir(image_folder_path):
        # 이미지 파일인지 확인 (일반적인 이미지 확장자)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            image_full_path = os.path.join(image_folder_path, filename)

            try:
                # 이미지 로드 및 RGB로 변환
                image = Image.open(image_full_path).convert("RGB")

                # 이미지 전처리 및 모델 예측
                inputs = processor(images=image, return_tensors="pt")
                inputs = {k: v.to(device) for k, v in inputs.items()}

                with torch.no_grad():
                    outputs = model(**inputs)
                logits = outputs.logits

                predicted_class_idx = logits.argmax(-1).item()
                predicted_label = model.config.id2label[predicted_class_idx]

                output_food_folder = os.path.join(output_base_path, predicted_label)
                if not os.path.exists(output_food_folder):
                    os.makedirs(output_food_folder)

                # 원본 이미지를 예측된 음식 종류 폴더로 복사
                shutil.copy(image_full_path, os.path.join(output_food_folder, filename))

            except Exception as e:
                print(f"오류 발생 ({filename}): {e}")

my_image_folder = "/content/drive/MyDrive/ColabNotebooks/MyFoodPhotos_Example" # 경로 수정하기
output_food_classified_folder = "/content/drive/MyDrive/Food101xOutput" # 경로 수정하기
classify_and_organize_food_images(my_image_folder, output_food_classified_folder)

def export_classified_images_to_csv(output_base_path, export_path_csv="classified_results.csv", export_path_xlsx=None):
    records = []

    for label in os.listdir(output_base_path):
        class_folder = os.path.join(output_base_path, label)
        if os.path.isdir(class_folder):
            for image_file in os.listdir(class_folder):
                if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    full_path = os.path.join(class_folder, image_file)
                    records.append({
                        "image_filename": image_file,
                        "predicted_label": label,
                        "full_path": full_path
                    })

    if records:
        df = pd.DataFrame(records)
        df.to_csv(export_path_csv, index=False)
        print(f"CSV 파일 저장 완료: {export_path_csv}")

        if export_path_xlsx:
            try:
                df.to_excel(export_path_xlsx, index=False)
            except ImportError:
                print("XLSX 파일 저장 실패")
    else:
        print("XLSX 파일 저장 실패")

export_classified_images_to_csv(
    output_base_path=output_food_classified_folder,
    export_path_csv="/content/drive/MyDrive/food101x_classified_summary.csv", # Drive에 저장될 CSV 파일
    export_path_xlsx="/content/drive/MyDrive/food101x_classified_summary.xlsx" # Drive에 저장될 XLSX 파일
)
print("완료")